# What do I mean by DSL Algebra?

When I say algebra, I don't think about implementing i.e. DSL for linear algebra in Python. I think about algebra in general as in "a set of objects and a collection of operations on them".

I like algebras, when they are used in programming. Probably the most well-known is Relational Algebra. The object in Relational Algebra is a *relation*, with operations like *join*, *projection* and *union*. The nice thing about having something that works like an algebra is, that you always work with the same type of object, reusing the box of tools without breaking the flow.

That is one of the reasons I like Pythons collections. Pythons sets and list actually have quite a nice algebra built around them.


In [2]:
{1,2,3} | {3,4,5}

{1, 2, 3, 4, 5}

In [3]:
{1,2,3} & {3,4,}

{3}

In [4]:
{1,2,3} - {3,4,5}

{1, 2}

And what I really like is, that all the laws I remember about sets from discrete math still work. For example, with union as `|` and empty set as `set()`:

In [5]:
a = {1,2,3}
a == a|set()==set()|a


True

In [6]:
b = {3,4,5}
c = {5,6,7}
(a | b) | c == a | (b | c)

True

My personal preference is to use these math-like operators over calling methods on objects, even though methpd calls would produce the same result, and often would be more efficient

In [7]:
a = {1,2}
a.add(3)
a.add(4)
a.remove(2)
a == ({1,2} | {3} | {4}) - {2}

True

To be honest, I probably would write this code with method calls, because they are more readable than operators: 

In [8]:
a == {1,2,3}.union({3,4}).difference({2})

True

I don't really want to have operators everywhere, what I want is composability. Now-days we might call this *fluent* interface, but I want to go one step further, I want the interface to conform to predefined set of rules.

This is what I mean when I say algebra:
* interfaces I work with *compose* with each other, they interoperate well, they are *fluent*
* the implementation is *lawful*, it conforms to predefined rules

By **dsl algebra** I mean two distinct, but related things:
* making a dsl that forms an algebra
* creating algebra that helps with creating dsl-s

# Why?

So I have been watching too many talks from Scala conferences and reading too many posts from haskell enthusiasts.
And now I envy them. Because I am mostly working with javascript and python and barely anybody even knows what `monad` *is*, while on the scala-side they are using co-free co-monads to implement cool, extensible DSLs.

Ok, there is fantasy-land and Ramda, but last time my colleague was trying to show off Ramda to the rest of the team, he mostly got puzzled looks from the most of them. And there isn't much in Python.

But I still want to try all of those cool things I see in the typed functional languages.

# Monoids are cool

One of the most useful concept to use here is the one of a *monoid*. 
If we think of this as an interface that is implemented by a type *T*, it needs two things:
* the *empty* element
* the *append* operation, that takes any two T's and returns a new T

There are two laws:
* for any a of type T: append(a,empty) == append(empty,a) == a
* for any a,b,c of type T: append(a,append(b,c)) == append(append(a,b),c)

Now we have this interface, we can do cool things with this :)

And by cool thigs, I mean going through [this presentation](https://www.youtube.com/watch?v=WsA7GtUQeB8) by Gabriel Gonzalez and translating the concepts from haskell to python :)

If you are not scared of Haskell, I really recomend that presentation. There is even a [souce for it](https://github.com/Gabriel439/slides/tree/master/lambdaconf/category).

Typeclasses present the first translation hiccup. In haskell you use these to signidy that a type conforms to some interface, similarily to Java/C# interfaces. Nice thing about this is, that you can define implementations for existing types. This makes them more similiar to C# extension methods, or Clojure protocols. Fortunately, we have single dispatch in Python to simulate this.

In [9]:
from functools import singledispatch
@singledispatch
def mempty(a):
    raise Error("Not implemented for" + a)

@singledispatch
def mappend(a, b):
    raise Error("Not implemented for" + a)

In [10]:
@mempty.register(list)
def _(a):
    return []

@mappend.register(list)
def _(a,b):
    return a + b

In [11]:
mappend([1,2,3],[4,5,6])

[1, 2, 3, 4, 5, 6]

In [12]:
@mempty.register(None.__class__)
def _(a):
    return None

@mappend.register(None.__class__)
def _(a,b):
    return None

In [13]:
mappend(None,None) == None

True

# Generic functions

Now we can create a generic function that works on all monoids, such as mconcat, that takes a list and appends its contents.

In [14]:
def mconcat(l):
    acc = l[0]
    for x in l[1:]:
        acc = mappend(acc,x)
    return acc

mconcat([[1,2],[3,4,5]])

[1, 2, 3, 4, 5]

As you can see, we have run into our first problem. In theory we should be able to do mconcat of an [] and get the mempty for the lists member type. But python doesn't have typed lists. Well, we see how far will this get us :)

# Nesting

Second thing we could do, is to try to nest these inside of other structures.
For example, if you have n-tuple with monoids, you can prove, that the n-tuple is monoid. Well, in Python you can't implement the (), but there is algebraic thing that behavesnjuat like monoid without the mempty element, called semi-group, so lets talk about these instead :)

In [16]:

@mappend.register((0,0).__class__)
def _(a,b):
    return tuple(mappend(i,j) for i,j in zip(a,b))

In [17]:
mappend(([1,2,3],[10,11,12]),([4,5,6],[14,15]))

([1, 2, 3, 4, 5, 6], [10, 11, 12, 14, 15])

In [18]:
mappend(([1,2],([10,11],([20,21],[25,26]))),([3,4],([12,13],([22,23],[26,28]))))

([1, 2, 3, 4], ([10, 11, 12, 13], ([20, 21, 22, 23], [25, 26, 26, 28])))

In [23]:
@mappend.register("".__class__)
def _(a,b):
    return a+b

# Functions as semigroups

If we have function f that accepts type A as input param and returns type B, then f forms a semigroup if B forms a semigroup.

In [24]:
@mappend.register(mconcat.__class__)
def _(a,b):
  def result(*x):
    a_r= a(*x)
    b_r=b(*x)
    return mappend(r_a,r_b)
  return result

In [25]:
a = input("asdf")


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.